<a href="https://colab.research.google.com/github/SEproject-Medihair/AI/blob/main/%ED%83%88%EB%AA%A8%EB%A0%88%EB%B2%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install efficientnet_pytorch


In [2]:
from google.colab import files

# 이미지 파일 및 JSON 파일 업로드
uploaded_files = files.upload()



Saving 6.탈모.zip to 6.탈모 (2).zip


In [3]:
from google.colab import files

# 이미지 파일 및 JSON 파일 업로드
uploaded_files = files.upload()

Saving 6.탈모.zip to 6.탈모 (3).zip


In [ ]:
zip_file_path = '6.탈모.zip'


In [5]:
import zipfile

# ZIP 파일의 경로 설정
zip_file_path = '6.탈모.zip'

# ZIP 파일 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/train_data/model6')




In [6]:
data_path = '/content/train_data/model6'


In [7]:
import time
import datetime
import os
import copy
import cv2
import random
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from PIL import Image
from efficientnet_pytorch import EfficientNet

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
hyper_param_batch = 6

random_seed = 100
random.seed(random_seed)
torch.manual_seed(random_seed)

num_classes = 4
model_name = 'efficientnet-b7'

train_name = 'model6'

PATH = './scalp_weights/'

data_path = './train_data/' + train_name

image_size = EfficientNet.get_image_size(model_name)
print(image_size)

600


In [8]:
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes)
model = model.to(device)

transforms_train = transforms.Compose([
    transforms.Resize([int(600), int(600)], interpolation=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Lambda(lambda x: x.rotate(90)),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data_set = datasets.ImageFolder(data_path, transform=transforms_train)

dataloaders, batch_num = {}, {}


Loaded pretrained weights for efficientnet-b7


In [9]:
dataloaders['train'] = DataLoader(train_data_set,
                                  batch_size=hyper_param_batch,
                                  shuffle=True,
                                  num_workers=4)

batch_num['train'] = len(train_data_set)

print('batch_size : %d, train: %d' % (hyper_param_batch, batch_num['train']))

class_names = train_data_set.classes
print(class_names)


batch_size : 6, train: 1154
['0.양호', '1.경증', '2.중등도', '3.중증']


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    start_time = time.time()

    since = time.time()
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    train_loss, train_acc = []


In [11]:
num_epochs = 1000
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    epoch_start = time.time()

    for phase in ['train']:
        model.train()

        running_loss = 0.0
        running_corrects = 0
        num_cnt = 0


Epoch 0/999
----------
Epoch 1/999
----------
Epoch 2/999
----------
Epoch 3/999
----------
Epoch 4/999
----------
Epoch 5/999
----------
Epoch 6/999
----------
Epoch 7/999
----------
Epoch 8/999
----------
Epoch 9/999
----------
Epoch 10/999
----------
Epoch 11/999
----------
Epoch 12/999
----------
Epoch 13/999
----------
Epoch 14/999
----------
Epoch 15/999
----------
Epoch 16/999
----------
Epoch 17/999
----------
Epoch 18/999
----------
Epoch 19/999
----------
Epoch 20/999
----------
Epoch 21/999
----------
Epoch 22/999
----------
Epoch 23/999
----------
Epoch 24/999
----------
Epoch 25/999
----------
Epoch 26/999
----------
Epoch 27/999
----------
Epoch 28/999
----------
Epoch 29/999
----------
Epoch 30/999
----------
Epoch 31/999
----------
Epoch 32/999
----------
Epoch 33/999
----------
Epoch 34/999
----------
Epoch 35/999
----------
Epoch 36/999
----------
Epoch 37/999
----------
Epoch 38/999
----------
Epoch 39/999
----------
Epoch 40/999
----------
Epoch 41/999
----------
Ep

In [12]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
from torch.utils.data import DataLoader
dataloaders = {
    'train': DataLoader(train_data_set, batch_size=hyper_param_batch, shuffle=True, num_workers=4)
}





for inputs, labels in dataloaders[phase]:
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    with torch.set_grad_enabled(phase == 'train'):
        outputs = model(inputs)

        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)


In [ ]:
running_loss += loss.item() * inputs.size(0)
running_corrects += torch.sum(preds == labels.data)
num_cnt += len(labels)

if phase == 'train':
    scheduler.step()

epoch_loss = float(running_loss / num_cnt)
epoch_acc = float((running_corrects.double() / num_cnt).cpu() * 100)

train_loss.append(epoch_loss)
train_acc.append(epoch_acc)

print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))


In [ ]:
if epoch_acc > best_acc:
    best_idx = epoch
    best_acc = epoch_acc
    best_model_wts = copy.deepcopy(model.state_dict())
    print('==> best model saved - %d / %.1f' % (best_idx, best_acc))


In [ ]:
epoch_end = time.time() - epoch_start

print('Training epochs {} in {:.0f}m {:.0f}s'.format(epoch, epoch_end // 60, epoch_end % 60))
print()


In [ ]:
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best valid Acc: %d - %.1f' % (best_idx, best_acc))

model.load_state_dict(best_model_wts)

torch.save(model, PATH + 'aram_' + train_name + '.pt')
torch.save(model.state_dict(), PATH + 'president_aram_' + train_name + '.pt')
print('model saved')

end_sec = time.time() - start_time
end_times = str(datetime.timedelta(seconds=end_sec)).split('.')
end_time = end_times[0]
print("end time :", end_time)


In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

num_epochs = 1000
train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=num_epochs)
